# Módulo 07, Tarefa 01

Vamos começar a mexer na nossa base de projeto? Já fizemos um exercício de montar a variável resposta, já pudemos perceber que essa atividade pode não ser trivial. Vamos agora trabalhar a base para que fique propícia ao *scikitlearn* para trabalharmos.

Lembrando, a base se chama demo01.csv, e originalmente está publicada [aqui](https://www.kaggle.com/rikdifos/credit-card-approval-prediction).

#### 1) Carregue a base e avalie:

- As variáveis
- Tipos de dados de cada variável
- Quantidade de missings
- Distribuição da variável resposta (mau)

In [1]:
# Importando as bibliotecas
import pandas as pd

from joblib import load, dump

In [2]:
# Importando os dados
link = "https://raw.githubusercontent.com/gfsilveira/ebac/main/"
link += "03-Cientista%20de%20Dados/02-Crisp-DM/M%C3%B3dulo_07/demo01.csv"
df = pd.read_csv(link)
df.head()

,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,possui_celular,possui_fone_comercial,possui_fone,possui_email,qt_pessoas_residencia,mau
0,M,Y,Y,0,Working,Secondary / secondary special,Married,House / apartment,58.832877,3.106849,1,0,0,0,2.0,False
1,F,N,Y,0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,52.356164,8.358904,1,0,1,1,1.0,False
2,F,N,Y,0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,52.356164,8.358904,1,0,1,1,1.0,False
3,M,Y,Y,0,Working,Higher education,Married,House / apartment,46.224658,2.106849,1,1,1,1,2.0,False
4,F,Y,N,0,Working,Incomplete higher,Married,House / apartment,29.230137,3.021918,1,0,0,0,2.0,False


In [3]:
# Analisando as variáveis
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16650 entries, 0 to 16649
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   sexo                   16650 non-null  object 
 1   posse_de_veiculo       16650 non-null  object 
 2   posse_de_imovel        16650 non-null  object 
 3   qtd_filhos             16650 non-null  int64  
 4   tipo_renda             16650 non-null  object 
 5   educacao               16650 non-null  object 
 6   estado_civil           16650 non-null  object 
 7   tipo_residencia        16650 non-null  object 
 8   idade                  16650 non-null  float64
 9   tempo_emprego          16650 non-null  float64
 10  possui_celular         16650 non-null  int64  
 11  possui_fone_comercial  16650 non-null  int64  
 12  possui_fone            16650 non-null  int64  
 13  possui_email           16650 non-null  int64  
 14  qt_pessoas_residencia  16650 non-null  float64
 15  ma

#### 2) Vamos montar um metadados

1. Crie um dataframe com os nomes de cada variável e o tipo de dados de cada variável.
2. Adicione uma coluna nesse *dataframe* chamada "qtd_categorias" e coloque nela o número de categorias correspondente de cada variável.
    Dica:
        1. inicie uma lista vazia
        2. faça um for ao longo dos nomes das variáveis,
        3. conte o número de categorias dessa variável
        4. acumule essa informação de 3. na lista que você criou em 1.
        5. No final, essa lista pode ser atribuída à nossa variável.
3. Crie variáveis dummy para as variáveis necessárias (i.e. aquelas que são qualitativas e não estão armazenadas como {0, 1} ou {True, False}.

In [4]:
def analise_variaveis(df: pd.DataFrame) -> pd.DataFrame:
  '''
      Função que analisa e retorna as carcateristicas das variáveis de uma
      DataFrame.
      :param df (pd.DataFrame): DataFrame à ser analisadas.
      :return df_dict_variaveis (pd.DataFrame): Informações sobre nome, tipo e
                                                quantidade de categorias
  '''
  # Copiando a DataFrame
  df_copy = df.copy()

  # Buscando os valores únicos em todas as variáveis
  df_variaveis = df_copy.apply(pd.unique).to_frame()

  # Buscando a quantidade de categorias
  df_variaveis = df_variaveis[0].apply(len).to_frame()

  # Reinicializar o index e inserindo como coluna
  df_variaveis.reset_index(inplace=True)

  # Instanciando os tipos (classes) das variáveis
  df_dict_variaveis = df_copy.dtypes.to_frame().reset_index()

  # Mesclar as informações de quantidade e tipo
  df_dict_variaveis = df_dict_variaveis.merge(df_variaveis, on="index")

  # Rennomeando as colunas
  df_dict_variaveis.rename(columns={
      "index": "variaveis",
      "0_x": "tipo",
      "0_y": "qtd_categorias"}, inplace=True)

  return df_dict_variaveis

In [5]:
# Analisando as variáveis do banco de dados
analise_variaveis(df)

,variaveis,tipo,qtd_categorias
0,sexo,object,2
1,posse_de_veiculo,object,2
2,posse_de_imovel,object,2
3,qtd_filhos,int64,8
4,tipo_renda,object,5
5,educacao,object,5
6,estado_civil,object,5
7,tipo_residencia,object,6
8,idade,float64,5298
9,tempo_emprego,float64,3005


#### 3) Crie variáveis dummy para as variáveis necessárias (i.e. aquelas que são qualitativas e não estão armazenadas como {0, 1} ou {True, False}. Crie um *dataframe* apenas com as variáveis apropriadas para entrada no scikitlearn - elimine as variáveis tipo *str*, mantendo apenas suas versões *dummy*.

In [6]:
# Selecionando as variáveis do tipo string (object) com mais de 2 categorias
filtro = (analise_variaveis(df).tipo == "object") & \
          (analise_variaveis(df).qtd_categorias > 2)

# Copiando o subconjunto com o filtro acima
df_dummy = df[analise_variaveis(df)[filtro]['variaveis'].values].copy()

# Obtendo os valores dummy das variáveis selecionadas
df_dummy = pd.get_dummies(df_dummy, drop_first=True)

# Obtendo os valores das variáveis não selecionadas
df_not_dummy = df[analise_variaveis(df)[~filtro]['variaveis'].values].copy()

# Mesclando os conjuntos de dados
df_dummyed = pd.concat([df_not_dummy, df_dummy], axis=1)

# Padronizando os valores
df_dummyed[df_dummyed.columns[:3]] = df_dummyed[df_dummyed.columns[:3]].replace(
    to_replace={"M": 0,"F": 1,"Y": 1,"N": 0})

df_dummyed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16650 entries, 0 to 16649
Data columns (total 29 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   sexo                                    16650 non-null  int64  
 1   posse_de_veiculo                        16650 non-null  int64  
 2   posse_de_imovel                         16650 non-null  int64  
 3   qtd_filhos                              16650 non-null  int64  
 4   idade                                   16650 non-null  float64
 5   tempo_emprego                           16650 non-null  float64
 6   possui_celular                          16650 non-null  int64  
 7   possui_fone_comercial                   16650 non-null  int64  
 8   possui_fone                             16650 non-null  int64  
 9   possui_email                            16650 non-null  int64  
 10  qt_pessoas_residencia                   16650 non-null  fl

In [7]:
# Analisando as variáveis do banco de dados
analise_variaveis(df_dummyed)

,variaveis,tipo,qtd_categorias
0,sexo,int64,2
1,posse_de_veiculo,int64,2
2,posse_de_imovel,int64,2
3,qtd_filhos,int64,8
4,idade,float64,5298
5,tempo_emprego,float64,3005
6,possui_celular,int64,1
7,possui_fone_comercial,int64,2
8,possui_fone,int64,2
9,possui_email,int64,2


In [8]:
# Deletando a variável possui_celular, pois todas as instâncias têm o mesmo valor
del df_dummyed['possui_celular']
df_dummyed.head()

,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,idade,tempo_emprego,possui_fone_comercial,possui_fone,possui_email,qt_pessoas_residencia,...,educacao_Secondary / secondary special,estado_civil_Married,estado_civil_Separated,estado_civil_Single / not married,estado_civil_Widow,tipo_residencia_House / apartment,tipo_residencia_Municipal apartment,tipo_residencia_Office apartment,tipo_residencia_Rented apartment,tipo_residencia_With parents
0,0,1,1,0,58.832877,3.106849,0,0,0,2.0,...,1,1,0,0,0,1,0,0,0,0
1,1,0,1,0,52.356164,8.358904,0,1,1,1.0,...,1,0,0,1,0,1,0,0,0,0
2,1,0,1,0,52.356164,8.358904,0,1,1,1.0,...,1,0,0,1,0,1,0,0,0,0
3,0,1,1,0,46.224658,2.106849,1,1,1,2.0,...,0,1,0,0,0,1,0,0,0,0
4,1,1,0,0,29.230137,3.021918,0,0,0,2.0,...,0,1,0,0,0,1,0,0,0,0


In [9]:
# Comparando os dados da variável email, no conjunto original e no dummy
valor = 'possui_email'
display(df_dummyed[valor].value_counts())
display(df[valor].value_counts())

0    15170
1     1480
Name: possui_email, dtype: int64

0    15170
1     1480
Name: possui_email, dtype: int64

In [10]:
# Comparando os dados da variável veículo, no conjunto original e no dummy
valor = 'posse_de_veiculo'
display(df_dummyed[valor].value_counts())
display(df[valor].value_counts())

0    10178
1     6472
Name: posse_de_veiculo, dtype: int64

N    10178
Y     6472
Name: posse_de_veiculo, dtype: int64

#### 4) Qual variável é mais poderosa?

Considere as variáveis ```possui_email``` e ```posse_de_veiculo```. Faça uma tabela cruzada entre elas e responda qual delas te parece mais poderosa para prever a probabilidade de ```mau = 1```?

In [11]:
# Analisando ambas as variáveis
pd.crosstab(df_dummyed['posse_de_veiculo'],df_dummyed['possui_email'])

possui_email,0,1
posse_de_veiculo,,
0,9341,837
1,5829,643


In [13]:
# Analisando email em relação ao pagamento
pd.crosstab(df_dummyed['possui_email'],df_dummyed['mau'])

mau,False,True
possui_email,,
0,14810,360
1,1450,30


In [12]:
# Analisando veículo em relação ao pagamento
pd.crosstab(df_dummyed['posse_de_veiculo'],df_dummyed['mau'])

mau,False,True
posse_de_veiculo,,
0,9931,247
1,6329,143


Comparando posse de veículo com e-mail, não observamos diferenças importantes, pois a distribuição entre quem possuí e não possuí, em ambas, é semelhante. Para quem não possuí veículo, ~90% não possuí e-mail, já quem possuí veículo, ~90% também não possuí e-mail.<br>

Nesse caso, é interessante comparar diretamente cada uma das variáveis com a variável dependente, maus. Observa-se que a grande maioria dos clientes não possuí e-mail e são bons pagadores, porém muito poucos deles (apenas 30) possuí e-mail e são maus pagadores. Portanto, essas duas variáveis parecem descorrelacionadas.<br>

Já a variável veículo está melhor distribuída, com dados para maus e bons pagadores com menores diferenças entre quem possuí e não veículo.<br>

Assim, a variável mais poderosa é a posse de veículo.<br>

#### 5) Salve a base, pois ela será utilizada no final deste módulo.

In [14]:
# Salvando o conjunto de dados
dump(df_dummyed, filename="df_dumyed")

['df_dumyed']

In [15]:
# Recuperando o conjunto de dados, verificando o salvamento
load("df_dumyed").head()

,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,idade,tempo_emprego,possui_fone_comercial,possui_fone,possui_email,qt_pessoas_residencia,...,educacao_Secondary / secondary special,estado_civil_Married,estado_civil_Separated,estado_civil_Single / not married,estado_civil_Widow,tipo_residencia_House / apartment,tipo_residencia_Municipal apartment,tipo_residencia_Office apartment,tipo_residencia_Rented apartment,tipo_residencia_With parents
0,0,1,1,0,58.832877,3.106849,0,0,0,2.0,...,1,1,0,0,0,1,0,0,0,0
1,1,0,1,0,52.356164,8.358904,0,1,1,1.0,...,1,0,0,1,0,1,0,0,0,0
2,1,0,1,0,52.356164,8.358904,0,1,1,1.0,...,1,0,0,1,0,1,0,0,0,0
3,0,1,1,0,46.224658,2.106849,1,1,1,2.0,...,0,1,0,0,0,1,0,0,0,0
4,1,1,0,0,29.230137,3.021918,0,0,0,2.0,...,0,1,0,0,0,1,0,0,0,0
